In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags

# Time-stepping
t_i = 0
t_f = -5000  # Run model for the past 5000 years
d_t = 0.5  # Use time step of 0.5 years
t = np.arange(t_f, t_i + 1, d_t)
t_kyr = t / 1000  # Years in kyr, needed to calculate orbital parameters

# Grid spacing
x_i = 0  # Northern boundary
x_f = 400000  # Southern boundary
d_x = 10000
x = np.arange(x_i, x_f + d_x, d_x)

# Parameters
A = 5.77e-4  # meters^-3 years^-1
alpha = 5
beta = 2

# Initialize glacier
h_ini = np.linspace(10, 0, len(x))

def D(H_vec):
    Diff = np.zeros_like(H_vec)
    for i in range(1, len(H_vec) - 1):
        Diff[i] = (
            abs(H_vec[i + 1] - H_vec[i - 1]) ** beta * (H_vec[i] ** alpha) / (2 * d_x) ** beta
        )
    Diff[0] = 1e-5  # Small value at boundaries
    Diff[-1] = 1e-5
    return Diff

def Alpha(D_vec):
    Alpha_diag = np.zeros(len(D_vec) - 1)
    for i in range(1, len(D_vec) - 1):
        Alpha_diag[i - 1] = (
            d_t
            / (4 * d_x ** 2)
            * (-D_vec[i + 1] + D_vec[i - 1] + 4 * D_vec[i])
            * A
            * 0.7 ** (beta + 1)
        )
    return Alpha_diag

def Beta(D_vec):
    Beta_diag = np.zeros(len(D_vec))
    for i in range(len(D_vec)):
        Beta_diag[i] = (
            d_t
            * ((-2 * D_vec[i]) / (d_x ** 2) + (1 / d_t))
            * A
            * 0.7 ** (beta + 1)
        )
    Beta_diag[0] = 1
    Beta_diag[-1] = 1
    return Beta_diag

def Gamma(D_vec):
    Gamma_diag = np.zeros(len(D_vec) - 1)
    for i in range(1, len(D_vec) - 1):
        Gamma_diag[i - 1] = (
            d_t
            / (4 * d_x ** 2)
            * (D_vec[i + 1] - D_vec[i - 1] + 4 * D_vec[i])
            * A
            * 0.7 ** (beta + 1)
        )
    return Gamma_diag

sols = np.zeros((len(x), len(t)))
D_ini = D(h_ini)
diffs = np.zeros((len(x), len(t)))
sols[:, 0] = h_ini
diffs[:, 0] = D_ini
h = h_ini

for i in range(len(t) - 1):
    # Solve for diffusivity value at each grid point
    Diffusivity = D(h)
    diffs[:, i] = Diffusivity

    # Calculate entries in P matrix
    lower_diag = Alpha(Diffusivity)
    center_diag = Beta(Diffusivity)
    upper_diag = Gamma(Diffusivity)

    # Build P matrix to solve
    diagonals = [np.append(lower_diag, 0), center_diag, np.insert(upper_diag, 0, 0)]
    M = diags(diagonals, [-1, 0, 1]).toarray()
    h[0] = 0
    h[-1] = 0

    h_new = np.linalg.solve(M, h)

    h = h_new
    # Update solutions
    sols[:, i + 1] = h

# Plot results
for i in range(4):
    plt.figure()
    plt.plot(diffs[:, i], label=f'Diffusivity at t={i}')
    plt.legend()

for i in range(4):
    plt.figure()
    plt.plot(sols[:, i], label=f'Height at t={i}')
    plt.legend()

plt.show()


ValueError: Diagonal length (index 1: 41 at offset 0) does not agree with matrix size (42, 42).

array([0.        , 9.79591837, 9.59183673, 9.3877551 , 9.18367347,
       8.97959184, 8.7755102 , 8.57142857, 8.36734694, 8.16326531,
       7.95918367, 7.75510204, 7.55102041, 7.34693878, 7.14285714,
       6.93877551, 6.73469388, 6.53061224, 6.32653061, 6.12244898,
       5.91836735, 5.71428571, 5.51020408, 5.30612245, 5.10204082,
       4.89795918, 4.69387755, 4.48979592, 4.28571429, 4.08163265,
       3.87755102, 3.67346939, 3.46938776, 3.26530612, 3.06122449,
       2.85714286, 2.65306122, 2.44897959, 2.24489796, 2.04081633,
       1.83673469, 1.63265306, 1.42857143, 1.2244898 , 1.02040816,
       0.81632653, 0.6122449 , 0.40816327, 0.20408163, 0.        ])